In step 1, I saved the molecules as RDKit object with 3D coordinates. This will enable the use of fingerprints like GETAWAY, MORSE, and USR that require those 3D coordinates.
<p>
For generating features, I will re-use a PyTorch Dataset subclass I had written for a previous project. Given a list of molecules, it generates values for most descriptors implemented in RDKit on first access to that molecule and cashes them for later reuse. Original version <a href="https://github.com/zetadin/TransferLearningFromPLS/blob/main/computeDescriptors.py">here</a>. This Dataset subclass also supports normalization across the whole dataset, calculation of only the specified descriptor groups, and filtering of features via  a mask.
<p>
In my experience, including 3D descriptors comptued from docked structures in the features to be trained on improves predictive power of the resulting models. However, for this project I do not have the time or resources to dock the molecules, so I will test 3D descriptors from undocked molecules help.
<p>
The plan is to test 4 sets of molecular representations here:
<ul>
  <li>2D descriptors</li>
  <li>2D + 3D Descriptors</li>
  <li>Reduced dimentionality by PCA of the better of above</li>
  <li>Reduced dimentionality exctracted from PLS for same</li>
</ul> 
The last two are probably going to be more helpfull for overtraining in NNs than for other model types.

# Import the custom Dataset subclass

In [1]:
from tqdm import tqdm, trange
import pickle

from CustomMolDataset import CustomMolDataset, dataBlocks

# Load the molecules

In [2]:
with open("molecules_3D.pickle", 'rb') as f:
    ligs = pickle.load(f)

# How fast is the calculation of descriptors and which ones need 3D embedding?
Which groups of descriptors to use are set with representation_flags, a list of 0s and 1s indicating if group should be used. Here I turn them on one at a time to check if they work without 3D coordinates and how long they take.

In [3]:
from rdkit import Chem
from rdkit.Chem import rdmolops
import time
from contextlib import redirect_stderr
from io import StringIO

flags_no3D = [0]*(len(dataBlocks)) # flags to generate only 2D descriptors

# test_ligands to figure out which dataBlocks are 2D, sincce I don't remember
test_ligs=[]
for i in range(5):
    mol = Chem.MolFromSmiles(ligs[i].GetProp('ID'))
    mol = rdmolops.AddHs(mol)
    mol.SetProp("ID", ligs[i].GetProp('ID'))    # ID
    mol.SetProp("pIC50", ligs[i].GetProp('pIC50')) # pIC50
    test_ligs.append(mol)
    
# loop over the different blocks of descriptors CustomMolDataset knows about
for block in range(len(dataBlocks)):
    flags=[0]*(len(dataBlocks))
    flags[int(dataBlocks(block))]=1 # turn on only the flag for this particular block of features

    DB2D = CustomMolDataset(test_ligs,
                          representation_flags = flags,
                          normalize_x = False,   # do not normalize the features yet
                          use_hdf5_cache = False, # cache results to file
                         )
    DB3D = CustomMolDataset(ligs[0:5],
                          representation_flags = flags,
                          normalize_x = False,   # do not normalize the features yet
                          use_hdf5_cache = False, # cache results to file
                         )
    
    # determine if this is a 2D or a 3D descriptor group by catching errors in 
    lbl = "2D"
    start=time.time()
    with redirect_stderr(StringIO()): # silence stderr
        try:
            for i in range(5):
                _,_=DB2D[i]
        except (RuntimeError, ValueError):
            lbl = "3D"
            start=time.time() # restart timer with embedded molecules
            for i in range(5):
                _,_=DB3D[i]
    # stop timer
    t = time.time() - start
    # mark this block of descriptors as 2D
    if(lbl=="2D"):
        flags_no3D[int(dataBlocks(block))]=1
    # clean up
    del DB2D, DB3D, flags
    
    print(f"{dataBlocks(block).name:20s}\t{lbl}\t{t*1e3/5:.3f} ms")

MACCS               	2D	0.617 ms
rdkitFP             	2D	0.942 ms
minFeatFP           	2D	1504.012 ms
MorganFP2           	2D	0.130 ms
MorganFP3           	2D	0.111 ms
Descriptors         	2D	5.185 ms
EState_FP           	2D	1.033 ms
Graph_desc          	2D	0.607 ms
MOE                 	2D	0.207 ms
MQN                 	2D	0.043 ms
GETAWAY             	3D	0.477 ms
AUTOCORR2D          	2D	0.025 ms
AUTOCORR3D          	3D	0.030 ms
BCUT2D              	2D	0.385 ms
WHIM                	3D	0.057 ms
RDF                 	3D	0.245 ms
USR                 	3D	0.007 ms
USRCUT              	3D	0.061 ms
PEOE_VSA            	2D	0.018 ms
SMR_VSA             	2D	0.006 ms
SlogP_VSA           	2D	0.006 ms
MORSE               	3D	0.299 ms


# Calculate all the quick features and  cache them
"minFeatFP" is a minimal feature set from RDKit's example for configurable descriptors. It is slow. I'll skip it.
However, even without it progress is getting temporarily stuck on some molecules. Need to find out on which set of descriptors is responcible and why.<p>
GETAWAY Fingerprint is also slow for some molecules. Not visible above, but appears when scanning through the whole dataset below. Will skip them too.<p>
BCUT2D still fails due to lack of Gasteiger Partial Charge assignment for Zinc. Skip them too.

In [4]:
flags_no3D[int(dataBlocks["minFeatFP"])]=0
flags_no3D[int(dataBlocks["GETAWAY"])]=0
flags_no3D[int(dataBlocks["BCUT2D"])]=0

flags_w3D = [1]*(len(dataBlocks))
flags_w3D[int(dataBlocks["minFeatFP"])]=0
flags_w3D[int(dataBlocks["GETAWAY"])]=0
flags_w3D[int(dataBlocks["BCUT2D"])]=0

In [5]:
 # calculate and cache the descriptors group by group to figure out which ones are getting stuck
for block in range(len(dataBlocks)):
    # skip minFeatFP and GETAWAY, as they are the slow ones
    if(dataBlocks(block).name=="minFeatFP" or
       dataBlocks(block).name=="GETAWAY" or
       dataBlocks(block).name=="BCUT2D"):
        continue;
        
    flags=[0]*(len(dataBlocks))
    flags[int(dataBlocks(block))]=1 # turn on only the flag for this particular block of features
    DB = CustomMolDataset(ligs,
                      representation_flags = flags,
                      normalize_x = False,   # do not normalize the features yet
                      use_hdf5_cache = True, # cache results to file
                      name = "EGFR_set_all_features"
                     )
    
    for i in trange(0, len(ligs), desc=dataBlocks(block).name):
        _,_=DB[i]
        
    del DB

MORSE: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 4635/4635 [00:03<00:00, 1331.45it/s]


# How many features do we have now and how many are unique and finite?
Features area now saved to HDD. How many of them are actually usable?

In [20]:
import numpy as np

# Function that eliminates non-useful features
# Returns list of feature indeces to keep.
# These can be passed to CustomMolDataset as an X_filter to only see desired features
def buildFeatureFilter(dataset):
    allX=np.array([entry[0] for entry in dataset])
    print("Shape of all data:", allX.shape)

    # exclude features that are always constant
    keep_feature_indeces = np.where(np.logical_not(np.all(allX == allX[0,:], axis=0)))[0]
    print(f"Variable (non-constant) features: {len(keep_feature_indeces)}")

    #purge features containing nans
    keep_feature_indeces = keep_feature_indeces[np.all(np.isfinite(allX[:,keep_feature_indeces]), axis=0)]
    print(f"Finite (no nan or inf for any ligand) features: {len(keep_feature_indeces)}")

    # correlation
    cormat=np.corrcoef(allX[:,keep_feature_indeces].transpose())
    cormat -= np.tril(cormat) # remove self correlation and lower triangular
    cormat=np.abs(cormat) # absolute values

    # find any duplicate features. They will have correlation = 1 to another feature.
    cor_threshhold = 1.0
    high_cor_pairs = np.where(cormat>=cor_threshhold)
    to_remove = np.unique(high_cor_pairs[0]) # indeces to the keep_feature_indeces array
    print(f"Highly correlated (>={cor_threshhold}) features to remove: {len(to_remove)}")

    # remove the duplicate features
    keep_mask = np.ones(len(keep_feature_indeces), np.bool_)
    keep_mask[to_remove] = 0
    keep_feature_indeces = keep_feature_indeces[keep_mask]

    print(f"Keapt features: {len(keep_feature_indeces)}")
    return(keep_feature_indeces)


# run this for both 2D+3D and just 2D feature combos
print("2D+3D descriptors:")
DB = CustomMolDataset(ligs,
                  representation_flags = flags_w3D,
                  normalize_x = False,   # do not normalize the features yet
                  use_hdf5_cache = True, # cache results to file
                  name = "EGFR_set_all_features"
                 )
X_filt_w3D = buildFeatureFilter(DB)
del DB

# just 2D
print("\n\n2D descriptors only:")
DB = CustomMolDataset(ligs,
                  representation_flags = flags_no3D,
                  normalize_x = False,   # do not normalize the features yet
                  use_hdf5_cache = True, # cache results to file
                  name = "EGFR_set_all_features"
                 )
X_filt_no3D = buildFeatureFilter(DB)
del DB

2D+3D descriptors:
Shape of all data: (4635, 7680)
Variable (non-constant) features: 7532
Finite (no nan or inf for any ligand) features: 7520
Highly correlated (>=1.0) features to remove: 92
Keapt features: 7428


2D descriptors only:
Shape of all data: (4635, 6980)
Variable (non-constant) features: 6832
Finite (no nan or inf for any ligand) features: 6820
Highly correlated (>=1.0) features to remove: 80
Keapt features: 6740


## Save the filters

In [7]:
with open("X_filt_w3D.pickle", 'wb') as f:
    pickle.dump(X_filt_w3D, f)

with open("X_filt_no3D.pickle", 'wb') as f:
    pickle.dump(X_filt_no3D, f)